In [6]:
.libPaths("/home/senyaaa/BI-PRS/packages")
#.libPaths("/home/senyaaa/BI-PRS/packages") # change to ur folder
#.libPaths("/home/senyaaa/BI-PRS/packages")

library(eurostat)

In [4]:
surname <- "pogodin"
L <- nchar(surname)
K <- 4
M <- ((K + L) * 47) %% 11 + 1
print(M)

[1] 1


In [9]:
srch <- search_eurostat(pattern='Police-recorded offences by NUTS 3 regions')
srch

title,code,type,last.update.of.data,last.table.structure.change,data.start,data.end,values,hierarchy
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
Police-recorded offences by NUTS 3 regions,crim_gen_reg,dataset,05.12.2023,05.12.2023,2008,2021,216947,4
Police-recorded offences by NUTS 3 regions,crim_gen_reg,dataset,05.12.2023,05.12.2023,2008,2021,216947,4


In [10]:
id <- "crim_gen_reg"
data <- get_eurostat(id=id)

Table crim_gen_reg cached at /tmp/Rtmp994Xyo/eurostat/adb2a8f5bb5815758c4af1b2ad610318.rds



In [11]:
data

freq,unit,iccs,geo,TIME_PERIOD,values
<chr>,<chr>,<chr>,<chr>,<date>,<dbl>
A,NR,ICCS0101,AL,2008-01-01,88
A,NR,ICCS0101,AL,2009-01-01,82
A,NR,ICCS0101,AL,2010-01-01,118
A,NR,ICCS0101,AL,2011-01-01,124
A,NR,ICCS0101,AL,2012-01-01,126
A,NR,ICCS0101,AL,2013-01-01,107
A,NR,ICCS0101,AL,2014-01-01,98
A,NR,ICCS0101,AL,2015-01-01,54
A,NR,ICCS0101,AL,2016-01-01,71
